In [1]:
import sys
import ete3
import os
from os import walk
sys.path.insert(0,'../scripts/')
from classDeclarationsAndFunctions import RootedTree
from fileIO import ReadRootedTree
from MarkovModels import GenerateQForStationaryDistribution, Get11FreeRates
from math import floor, ceil
from config import projectPath
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# scripts for submitting to server 

# run tools
# iqtree
import numpy as np
# iqtree_script = open("../batch_scripts/iqtree_scalability.sh","w")
# mstbackbone_script = open("../batch_scripts/mstbackbone_scalability.sh","w")
# fasttree_script = open("../batch_scripts/fastree_scalability.sh","w")
# raxml_script = open("../batch_scripts/raxml_scalability.sh","w")
data_path = projectPath + "data/"
tools_path_geniux = "/project/exaptation/Tools/"
num_trees_simulated = 0 
for seq_len in [1000,2000,5000,10000]:
    iqtree_script = open("../batch_scripts/iqtree_scalability_"+str(seq_len)+".sh","w")
    mstbackbone_script = open("../batch_scripts/mstbackbone_scalability_"+str(seq_len)+".sh","w")
    fasttree_script = open("../batch_scripts/fastree_scalability_"+str(seq_len)+".sh","w")
    raxml_script = open("../batch_scripts/raxml_scalability_"+str(seq_len)+".sh","w")    
    tree_id_num_leaves_scalability_file = open("../data/selected_grove_tree_ids_scalability","r")
    for line in tree_id_num_leaves_scalability_file:        
        tree_id = line.strip().split(',')[0].strip()
        num_leaves = int(line.strip().split(',')[1].strip())
        num_trees_simulated += 1
        print("Writing batch commands for tree ", str(num_trees_simulated))    
        mxqsub_prefix = ""
        exp_id = "treeId_"+tree_id+"_numLeaves_"+str(num_leaves)+"_seqLen_"+str(seq_len)
        exp_dir = data_path + "grove_exp/" + exp_id
        sequence_file_name = exp_dir + "/sequences_"+exp_id+".fas"                
        ###############        IQ-TREE      ##################
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N iqtree_scale\t"
        # mxqsub_prefix += " --stderr=err_iqtree_scalability_"+exp_id
        # mxqsub_prefix += " --stdout=out_iqtree_scalability_"+exp_id
        iqtree_command = tools_path_geniux + "iqtree2"
        iqtree_command += ' -s ' + sequence_file_name + ' -st DNA'
        iqtree_command += ' -nt ' + str(1)
        iqtree_command += ' -seed 1234'
        # iqtreeCommand += ' -quiet'
        iqtree_command += ' -redo'
        iqtree_command += ' -pre ' + sequence_file_name + "_iqtree2.0"
        iqtree_command += ' -m 12.12+FO'
        iqtree_script.write(mxqsub_prefix + iqtree_command + "\n")
        ###############     MST-BACKBONE     ###################
        # mst_backbone_command
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N mstbackbone_scale\t"
        # mxqsub_prefix += " --stderr=err_mstbackbone_scalability_" + exp_id
        # mxqsub_prefix += " --stdout=out_mstbackbone_scalability_" + exp_id
        mst_backbone_command = tools_path_geniux + "mst-backbone"
        mst_backbone_command += '\t--seq ' + sequence_file_name
        mst_backbone_command += '\t--out ' + exp_id
        mstbackbone_script.write(mxqsub_prefix + mst_backbone_command + "\n")
        ###############        RAxML-NG       ##################
        # mxqsub_prefix = "mxqsub -t 2d -m 10G -n -N raxmlng_scale\t"
        # mxqsub_prefix += " --stderr=err_raxmlng_scalability_" + exp_id
        # mxqsub_prefix += " --stdout=out_raxmlng_scalability_" + exp_id
        raxml_command = tools_path_geniux + "raxml-ng"
        fileNamePrefix = 'RAxMLNG_' + exp_id
        raxml_command += ' --tree pars{1}'
        raxml_command += ' --model GTR --msa ' + sequence_file_name
        raxml_command += ' --threads 1'
        # raxml_command += ' --redo '
        raxml_command += ' --tip-inner on --pat-comp on --site-repeats on'
        raxml_command += ' --prefix ' + fileNamePrefix
        raxml_script.write(mxqsub_prefix + raxml_command + "\n")
        #############          FastTree        ################
        # mxqsub_prefix = ""
        # mxqsub_prefix = "mxqsub -t 2d -m 1G -n -N fasttree_scale\t"
        # mxqsub_prefix += " --stderr=err_fasttree_scalability_"+exp_id
        # mxqsub_prefix += " --stdout=out_fasttree_scalability_"+exp_id
        output_tree_file_name = sequence_file_name + '.fasttree_newick'
        std_err_file_name = sequence_file_name + '.fasttree_errlog'
        fasttree_command = tools_path_geniux + 'FastTree'
        fasttree_command += ' -nt -nosupport -nocat -gtr < '+ sequence_file_name
        fasttree_command += ' > ' + output_tree_file_name
        fasttree_command += ' 2> ' + std_err_file_name
        fasttree_script.write(mxqsub_prefix + fasttree_command + "\n")
        ########################################################
    tree_id_num_leaves_scalability_file.close()
    raxml_script.close()
    fasttree_script.close()
    mstbackbone_script.close()
    iqtree_script.close()

In [4]:
# results list (count num of results)
run_time_num_taxa = {}
run_time_seq_length = {}
data_path = projectPath + "data/"
tools_path_geniux = "/project/exaptation/Tools/"
# method_list = ['mstbackbone','fasttree','iqtree','raxmlng']
method_list = ['fasttree','iqtree','mstbackbone','raxmlng']
seqlen_list = [1000,2000,5000,10000]
def Check_results_file_names(method,exp_id):
    exp_dir = data_path + "grove_exp/" + exp_id
    sequence_file_name = exp_dir + "/sequences_" + exp_id + ".fas"
    if method == "raxmlng":
        log_file_name = projectPath + '/batch_scripts/RAxMLNG_' + exp_id + '.raxml.log'
        tree_file_name = projectPath + '/batch_scripts/RAxMLNG_' + exp_id + '.raxml.bestTree'
    elif method == "mstbackbone":
        # log_file_name = projectPath + '/batch_scripts/' + exp_id + '.mstbackbone_log'
        log_file_name = data_path + 'grove_exp/' + exp_id + '/' + exp_id + '.mstbackbone_log'
        tree_file_name = data_path + 'grove_exp/' + exp_id + '/' + exp_id + '.newick'
    elif method == "fasttree":
        log_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas.fasttree_errlog'
        tree_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas.fasttree_newick'
    elif method == "iqtree":
        log_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas_iqtree2.0.log'
        tree_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas_iqtree2.0.treefile'
    if os.path.exists(log_file_name) and os.path.exists(tree_file_name):
        if os.path.getsize(log_file_name) > 0 and os.path.getsize(tree_file_name) > 0:
            return(True)
        else:
            return(False)
    else:
        return(False)

num_results = {}
for method in method_list:
    num_results[method] = 0

tree_id_num_leaves_scalability_file = open(data_path + "selected_grove_tree_ids_scalability","r")
for line in tree_id_num_leaves_scalability_file:
    tree_id = line.strip().split(',')[0].strip()
    num_leaves = int(line.strip().split(',')[1].strip())
    num_trees_simulated += 1    
    for seq_len in seqlen_list:
        for method in method_list:            
            exp_id = "treeId_" + tree_id + "_numLeaves_" + str(num_leaves) + "_seqLen_" + str(seq_len)
            if Check_results_file_names(method,exp_id):
                num_results[method] += 1

tree_id_num_leaves_scalability_file.close()

for method in method_list:
    print (num_results[method], "results available for ", method)
            # exp_dir = data_path + "grove_exp/" + exp_id
            # sequence_file_name = exp_dir + "/sequences_" + exp_id + ".fas"
            
            
            

File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/data/grove_exp/treeId_12258_numLeaves_999_seqLen_1000/sequences_treeId_12258_numLeaves_999_seqLen_1000.fas.fasttree_errlog
File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/data/grove_exp/treeId_12258_numLeaves_999_seqLen_2000/sequences_treeId_12258_numLeaves_999_seqLen_2000.fas.fasttree_errlog
File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/data/grove_exp/treeId_12258_numLeaves_999_seqLen_5000/sequences_treeId_12258_numLeaves_999_seqLen_5000.fas.fasttree_errlog
File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/data/grove_exp/treeId_12258_numLeaves_999_seqLen_10000/sequences_treeId_12258_numLeaves_999_seqLen_10000.fas.fasttree_errlog
File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/data/grove_exp/treeId_11413_numLeaves_1074_seqLen_1000/sequences_treeId_11413_numLeaves_1074_seqLen_1000.fas.fasttree_errlog
File exists
/home/kalaghat/exaptation/Projects/MSTBasedForests/dat

In [ ]:
def Get_run_time(method,exp_id):
    if method == "raxmlng":
        log_file_name = projectPath + '/batch_scripts/RAxMLNG_' + exp_id + '.raxml.log'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Elapsed time:'):
                runTime = float(line.strip().split('Elapsed time:')[1].split(' seconds')[0])                
    elif method == "mstbackbone":        
        log_file_name = data_path + 'grove_exp/' + exp_id + '/' + exp_id + '.mstbackbone_log'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total CPU time used is'):
                runTime = float(line.split('Total CPU time used is ')[1].split(' second(s)')[0])
    elif method == "fasttree":
        log_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas.fasttree_errlog'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total time: '):
                runTime = float(line.strip().split('Total time: ')[1].split(' seconds')[0])
    elif method == "iqtree":
        log_file_name = data_path + 'grove_exp/' + exp_id + '/sequences_' + exp_id + '.fas_iqtree2.0.log'
        log_file = open(log_file_name,"r")
        for line in log_file:
            if line.startswith('Total CPU time used:'):
                runTime = float(line.split('Total CPU time used: ')[1].split(' seconds')[0])

    log_file.close()
    return (runTime)

runTimes = {}

tree_id_num_leaves_scalability_file = open(data_path + "selected_grove_tree_ids_scalability","r")
for line in tree_id_num_leaves_scalability_file:
    tree_id = line.strip().split(',')[0].strip()
    num_leaves = int(line.strip().split(',')[1].strip())
    num_trees_simulated += 1    
    for seq_len in seqlen_list:
        for method in method_list:            
            exp_id = "treeId_" + tree_id + "_numLeaves_" + str(num_leaves) + "_seqLen_" + str(seq_len)
            if Check_results_file_names(method,exp_id):
                num_results[method] += 1

tree_id_num_leaves_scalability_file.close()



In [ ]:
# Compute accuracy

In [ ]:
# scalability plot (sequence length)